# Visualizing and Understanding Convolutional Networks

    论文作者：Matthew D. Zeiler and Rob Fergus 
这是2014年ECCV上的经典文献，CNN领域可视化理解的开山之作


## 主旨：

介绍了新的可视化手段，可以查看CNN中不同的层学习到了什么特征，是怎样对总体做出贡献的，以及我们怎么通过可视化来调整网络以提高精度。\
这里的可视化手段指的就是deconvnet反卷积网络。

## 内容：

文中使用的模型输入为彩色2d图片经过层输出对应类别的可能性向量，每一层包括前一层的卷积（第一层为输入的数据），激活函数，池化与标准化（后两项为可选项）。
### deconvnet反卷积网络
什么是deconvnet反卷积网络：可以被认为是反向使用相同组件（过滤，池）的卷积模型。它在被提出时被认为是一种无监督学习方式，但在这里它没有学习能力，只是对已训练的卷积的探索。应用反卷积网络辅助理解一个卷积神经网络，需要在该网络模型的每一层附带一个反卷积层，在检测理解某一层的某一个activation时，我们需要先将该层的其他activation的值置为0，然后将该层的特征图作为输入传递给该层附带的反卷积层，依次执行unpooling, rectification, filtering 来重构输入像素特征。\
直觉解释：Since the model is trained discriminatively, they implicitly show which parts of the input image are discriminative. Note that these projections are not samples from the model, since there is no generative process involved. The whole procedure is similar to backpropping a single strong activation。（觉得写的很好，就不翻译了）

见图：                                                                                                                               

![title](img/unpooling.png)

![title](img/1.png)

我们具体看看unpooling, rectification, filtering 怎么操作：

#### 反向池化unpooling

池化是不可逆的，但我们可以记录池化过程中最大激活值的位置，然后在反池化的时候把这个位置的值激活，而其他值为0（只是一种近似，池化中不是真的为0）。举例：假设我们pooling块的大小是3×3，采用max pooling后，我们可以得到一个输出神经元其激活值为9，pooling是一个下采样的过程，3×3大小变成了1×1大小的图片了。而unpooling是一个上采样的过程，当我们由一个神经元要扩展到3×3个神经元的时候，我们需要借助于pooling过程中，记录下最大值所在的位置坐标(0, 1)，然后在unpooling过程的时候，就把(0, 1)这个像素点的位置填上去，其它的神经元激活值全部为0。                           

#### 反向激活rectification

卷积神经网络中使用ReLU,是为了保证特征图为正值。反向过程也需要这个保证，所以就直接也用ReLU函数。

#### 反向卷积filtering

对于反卷积过程，采用卷积过程转置后的滤波器(参数一样，只不过把参数矩阵水平和垂直方向翻转了一下)。

### 卷积网络可视化：

下图是将模型进行特征可视化的结果示意图。对于一个给定的特征图,我们选取前9个activations，每一个都单独反卷积过去投射到像素空间(投射的结果称为projections)，从而揭示了与之对应的filter所敏感的像素特征。可以发现这与人类视觉有很大不同。例如，在5层中，第一列的第二个中，图像块彼此并没有什么相同点，但相应的filter却发现了它们有相同的背景，并不是背景前面的物体。 \
此外，每一层的projections显示的信息均具有层次感，也体现了神经网络观察分辨图片的层次能力。如第二层网络对应的是边界、轮廓和色块信息；第三层捕捉的是纹理信息；第四层展现了较大的差异主要是一些类别上部分的信息，狗的脸(R1,C1)，鸟的腿(R4,C2)等；第五层展示的是不同姿势的整个物体的视图，比如键盘(R1,C11)和狗(R4)。\
而从不同层的特征图来看，在训练过程中，底层特征收敛较快，然而高层的特征要经过相当多次迭代才可以收敛。所以要想使得神经网络模型有好的性能，就要多次迭代使得它达到全网络层收敛。

![title](img/visualization.png)

### 结构改进
可视化同样可以帮助改进卷积网络的结构。以下是第一层和第二层改进前后的对比图：\
先观察改进前，可以看到第一层filter混合了高频和低频的信息，中间频率则很少被覆盖，第二层出现了混叠现象（由第一层步长过大导致的）
所以我们提出解决方案：
- 第一层采用更小的filter，尺寸从11 * 11到7 * 7
- 缩小stride间隔，从4变为2。

通过可视化可以有针对地改进结构，获取更多的信息，而且提升分类准确率（按照实验部分的数据，提高了1.7%）

![title](img/architecture.png)

### Occlusion sensitive 遮挡敏感性
对于图片分类算法来说，一个重要的问题是模型是真正地识别了图像中物体的位置，还是仅仅使用了周围的环境。本文用灰色方块进行了遮挡实验，证明模型确实识别了物体位置（挡住物体后正确率大跌）。
b是遮挡不同部分得到的feature map，c是在第5层 响应最高的feature map的projection，d是遮住不同部分得到的groundtruth所在类的score map，e是 score maps的分类结果。可以看出对于物体的关键部分遮挡之后会极大的影响分类结果。可以注意到第二个和第三个例子中分别是文字和人脸的响应更高，但是却不是关键部分。

![title](img/occlusion.png)

### 实验结论
- 模型深度对于模型性能很重要，存在一个最小深度，当小于此深度时，模型性能大幅下降。证据：去除掉包含大部分网络参数的最后两个全连接层之后，网络性能下降很少；去掉中间两个卷积层之后，网络性能下降也很少；但是当把他们都去掉之后，网络性能急剧下降。
- 改变全连接层的大小对结果影响不大；增加中间卷积层尺寸可以有效地提高性能（但这会导致全连接层的扩大并导致过拟合）。

### 特征的泛化能力
作者固定了原来网络的权值，只是使用新数据训练了softmax分类器，效果非常好。这说明卷积网络相当于一个通用的特征提取器。因为ImageNet数据量，类别多，所以由ImageNet训练出来的模型也可以很好地适用于其他数据集：在caltech101和caltech256上都达到了state-of-the-art的准确率；但是在pascal上却比当前最好成绩差3.2%（这或许是由于pascal数据集和ImageNet数据集有很大的不同，pascal数据集上有多个物体）。

## 想法：
我认为本文的贡献有三点：一是神经网络的可解释化，二是可视化改进模型，三是泛化的证明。\
神经网络往往被认为是一个黑箱，很多成果知其然不知其所以然，而文中的可视化和遮挡实验都是很好地解构神经网络，解释其原理的好方法，神经网络的可解释性是我们在追求效率的时候也应该追求的，可以指明发展的方向。可视化改进模型就是在可解释化的基础上产生的准确率的提升，这就是理论的完备带来的工业生产上的提升。而泛化的证明则是为工业界小数据集开辟了新的方向：可以寻找相似的大数据集训练大模型，再使用大模型来微调以适应小数据集，相信会比直接在小数据集上训练有很大的提升。

对于NLP领域而言想要应用CNN，输入可以是词向量，但长度是不固定的，所以只有一对卷积层和池化层（或是预处理成一样长度的）。所以有如下一些问题：
- CNN在NLP中似乎大部分比较浅，但文中证明了深度的重要性，那么我们是否应该在NLP中使用多层CNN？
- CNN会丢失一部分位置信息，那是否可以把RNN或者LSTM和CNN连起来用？
- 我们知道卷积的优越性在于抗平移，在NLP中应该也有同样的特征抓取能力。但文字信息并不如图片那么直观，那么怎么实现NLP中CNN的可解释性呢？（关于这个问题我在知乎上找到了一篇不错的文章：https://zhuanlan.zhihu.com/p/84318890 NLP中神经模型的可视化与理解。）
- 文中实现的CNN的泛化能力在NLP中也是一样的吗？

## 总结：
通过可视化手段可以看到，卷积层捕捉的特征虽然和人类视觉不同，但也有很强的层次和规律在其中。可视化手段也可以帮助我们改进模型。遮挡实验，特征泛化实验也证明了卷积神经网络模型是逻辑合理的通用特征提取器。\
文中的结论也提醒我们注意CNN模型的深度，卷积核大小和步长，以及中间卷积层的尺寸。